In [43]:
import numpy as np
import cv2

class Table:
    def __init__(self, bbox = (0,0,0,0), rows = 0, cols = 0, cells = None):
        self.bbox = bbox
        self.rows = rows
        self.cols = cols
        self.cells = cells
    def disp_table(self):
        print("No. of ROWS: {}".format(self.rows))
        print("No. of COLUMNS: {}".format(self.cols))
        print("No. of Cells: {}".format(len(self.cells)))
        print("Table Width: {}".format(self.bbox[2]))
        print("Table Height: {}".format(self.bbox[3]))
        print("\n\n")

class TextBox:
    def __init__(self, bbox=(0,0,0,0)):
        self.bbox = bbox
    def disp_textbox(self):
        x,y,w,h = self.bbox
        print("x = ",x)
        print("y = ",y)
        print("w = ",w)
        print("h = ",h)
        print("\n\n")

In [58]:
img1 = cv2.imread('emp_form.jpg')
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img_bin,ab1 = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
img_bin,ab2 = cv2.threshold(img, 244, 255, cv2.THRESH_BINARY)
ab = cv2.addWeighted(ab1, 0.6, ab2, 0.4, 0.0)
cv2.imwrite("RESULT2.jpg", ab)
ab = cv2.bitwise_not(ab)

In [59]:
horizontal = np.copy(ab)
# Specify size on horizontal axis
cols = horizontal.shape[1]
horizontal_size = cols // 30
# Create structure element for extracting horizontal lines through morphology operations
horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))
    # Apply morphology operations
horizontal = cv2.erode(horizontal, horizontalStructure)
horizontal = cv2.dilate(horizontal, horizontalStructure)

# Show extracted horizontal lines
cv2.imwrite("RESULT3.jpg",horizontal)

True

In [60]:
vertical = np.copy(ab)
# Specify size on vertical axis
rows = vertical.shape[0]
verticalsize = rows // 100
    # Create structure element for extracting vertical lines through morphology operations
verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))
    # Apply morphology operations
vertical = cv2.erode(vertical, verticalStructure)
vertical = cv2.dilate(vertical, verticalStructure)
ch = vertical | horizontal
#ch = cv2.bitwise_not(ch)
cv2.imwrite("RESULT3.jpg",ch)

True

In [61]:
contours, hierarchy = cv2.findContours(ch, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
img2 = np.copy(img1)
count = 0
boxes = list()
for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if  w>15 and h>15 and h<img2.shape[0]:
          cv2.rectangle(img2, (x, y), (x+w, y+h), (0,0, 255), 2)
          count+=1
          boxes.append((x,y,w,h))
print(count)
print(max(boxes, key = lambda x:x[3]))
boxes.sort(key = lambda x: x[3], reverse = True)
print(boxes)
cv2.imwrite("RESULT4.jpg",img2)
cv2.imshow("IMAGE",img2)
cv2.waitKey(0)

39
(10, 94, 718, 238)
[(10, 94, 718, 238), (11, 117, 716, 212), (10, 508, 718, 206), (11, 509, 716, 203), (10, 333, 718, 116), (10, 745, 718, 103), (10, 859, 718, 27), (583, 818, 144, 27), (372, 818, 211, 27), (225, 818, 146, 27), (11, 818, 214, 27), (583, 790, 144, 27), (372, 790, 211, 27), (225, 790, 146, 27), (11, 790, 214, 27), (583, 763, 144, 26), (372, 763, 211, 26), (225, 763, 146, 26), (11, 763, 214, 26), (10, 718, 718, 26), (10, 483, 718, 24), (645, 424, 82, 23), (529, 424, 115, 23), (225, 424, 189, 23), (415, 424, 114, 22), (11, 424, 214, 22), (645, 402, 82, 22), (529, 402, 115, 22), (415, 402, 114, 22), (225, 402, 189, 22), (11, 402, 214, 22), (645, 380, 82, 21), (529, 380, 115, 21), (415, 380, 114, 21), (225, 380, 189, 21), (11, 380, 214, 21), (10, 966, 40, 17), (10, 925, 40, 17), (10, 898, 42, 17)]


-1

In [62]:
def form_tables(boxes):
    k = 0
    i = 0
    tables = list()
    tot = len(boxes)
    while i < len(boxes):
        x,y,w,h = boxes[i][0:4]
        k += 1
        j = k
        temp = list()
        while j < len(boxes):
            if boxes[j][0] <= x+w  and boxes[j][1] <= y+h and x < boxes[j][0]  and y < boxes[j][1]  :
                temp.append(boxes[j])
                boxes.remove(boxes[j])
            else:    
                j += 1
        if len(temp) == 0:
            i += 1
            continue
        temp.sort(key = lambda z: z[0])
        #print(temp)
        cols = 1
        for u in range(len(temp)-1):
            if abs(temp[u][0] - temp[u+1][0]) <= 3:
                pass
            else:
                cols += 1
        temp.sort(key = lambda z: z[1])
        rows = 1
        for u in range(len(temp)-1):
            if abs(temp[u][1] - temp[u+1][1]) <= 3:
                pass
            else:
                rows += 1
        t = Table((x,y,w,h), rows, cols, temp)
        tables.append(t)
        i += 1
    return tables, boxes
            

#tables stores objects of Table class

temp_boxes  = boxes.copy()
tables, temp_boxes = form_tables(temp_boxes)

if len(tables)==0:
    print("No TABLES")
for i in range(len(tables)):
    print("TABLE ", i+1)
    tables[i].disp_table()
        

TABLE  1
No. of ROWS: 1
No. of COLUMNS: 1
No. of Cells: 1
Table Width: 718
Table Height: 238



TABLE  2
No. of ROWS: 1
No. of COLUMNS: 1
No. of Cells: 1
Table Width: 718
Table Height: 206



TABLE  3
No. of ROWS: 3
No. of COLUMNS: 5
No. of Cells: 15
Table Width: 718
Table Height: 116



TABLE  4
No. of ROWS: 3
No. of COLUMNS: 4
No. of Cells: 12
Table Width: 718
Table Height: 103





In [63]:
for i in range(len(tables)):
    if tables[i].bbox in temp_boxes:
        temp_boxes.remove(tables[i].bbox)
TEXT_BOXES = temp_boxes.copy()
print(TEXT_BOXES)
#TEXT_BOXS stores text boxes

[(10, 859, 718, 27), (10, 718, 718, 26), (10, 483, 718, 24), (10, 966, 40, 17), (10, 925, 40, 17), (10, 898, 42, 17)]


In [64]:
#text boxes
img3 = img2
for i in TEXT_BOXES:
    x,y,w,h = i[0:4]
    cv2.rectangle(img3, (x, y), (x+w, y+h), (255, 0, 0), 3)
cv2.imwrite("RESULT4.jpg", img3)

True

In [65]:
text_boxes = [TextBox(x) for x in TEXT_BOXES]
for i in range(len(text_boxes)):
    text_boxes[i].disp_textbox()
    
### in RESULT4.jpg blue boxes represent text boxes and red are tables
### 2 text boxs are identified as tables thats the error, but in ur code they get identified as sections so with that  
### there should be no errors at all

x =  10
y =  859
w =  718
h =  27



x =  10
y =  718
w =  718
h =  26



x =  10
y =  483
w =  718
h =  24



x =  10
y =  966
w =  40
h =  17



x =  10
y =  925
w =  40
h =  17



x =  10
y =  898
w =  42
h =  17



